In [1]:
import python_utils as utils
import sys
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
#%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# SIGMOIDPULSE0_002 t_mid = 13000 duration = 10 s

In [2]:
parseg = 500
neglect_t = 3000
t_mid = 13000
t_mid2 = t_mid + 10000
sigm_par = 0.002

In [3]:
t_mid_poiss = str(t_mid)
sigm_par_poiss = "{:.5f}".format(sigm_par)
function = "sigmoidpulse" + str(format(sigm_par, 'f')).replace('.', '_')
while function.endswith('0'):
    function = function[:-1]
Dd = "0.950"
subnets = "STN"
simulations = [str(i) for i in range(32)]

new_Dd = Dd.replace('.','_')


## funzione dell'intensità della sigmoide attesa

In [4]:
#parseg 250 0verlap 125
Dd_values = [0.85, 0.855, 0.86, 0.865, 0.87, 0.875, 0.88, 0.885, 0.89, 0.895, 0.9, 0.905, 0.91, 0.915, 0.92, 0.925, 0.93, 0.935, 0.94, 0.945, 0.95]
pow_t_mean_point = [0.019914247149334242, 0.020690578678315804, 0.02158271146066814, 0.022364914133701584, 0.023428835457957573, 0.025125497868107482, 0.026346934723841548, 0.028828969225659612, 0.03220798123872726, 0.035254298671665026, 0.03867622848014253, 0.04674626548972391, 0.05068805695941251, 0.06606716969376104, 0.08487862320152097, 0.09270422348347553, 0.1322581180913206, 0.15084846176010766, 0.18747180115646506, 0.22419232383889678, 0.2697145569231209]
pow_error_point = [0.0019838232642017555, 0.001999857551482696, 0.0021708086191144817, 0.0022749744417916227, 0.002441049902399858, 0.002640832617992426, 0.0027738692999452765, 0.0031292618798778368, 0.0036480613158226044, 0.0040683093611075605, 0.00451842724610424, 0.005668445395626774, 0.006082506534680942, 0.007877499532094852, 0.009638645329156629, 0.010410256249777311, 0.012819346780713003, 0.01291999850097032, 0.014568471805002196, 0.015395217530866883, 0.015333769105628054]


In [5]:
Dd_values = Dd_values[::-1] + Dd_values
pow_t_mean_point = pow_t_mean_point[::-1] + pow_t_mean_point
pow_error_point = pow_error_point[::-1] + pow_error_point

In [6]:
intermediate_points = [round((Dd_values[i] + Dd_values[i + 1]) / 2, 4) for i in range(len(Dd_values) - 1)]


In [7]:
# Inizializza variabile di controllo per il secondo loop
start_second_loop = False

t_d_int = []
t_d_int.append(33000)
# Primo loop
for x in intermediate_points[:]:
    # Controlla se è il momento di iniziare il secondo loop
    if x == 0.85:
        start_second_loop = True
        t_d_int.append((t_mid2 + t_mid)/2)
        continue

    # Calcola i valori in base alla variabile di controllo
    if not start_second_loop:
        y = -np.log((0.1) / (x - 0.85) - 1) / sigm_par + t_mid2
    else:
        y = -np.log((-0.1) / (x - 0.95) - 1) / sigm_par + t_mid

    # Aggiungi il valore calcolato a t_d_int
    t_d_int.append(y)

t_d_int.append(0)

t_d_int = np.array(t_d_int)
t_d_int = np.round(t_d_int).astype(int)


In [8]:
t_d_int

array([33000, 24832, 24256, 23973, 23775, 23618, 23485, 23365, 23255,
       23151, 23050, 22950, 22849, 22745, 22635, 22515, 22382, 22225,
       22027, 21744, 21168, 18000, 14832, 14256, 13973, 13775, 13618,
       13485, 13365, 13255, 13151, 13050, 12950, 12849, 12745, 12635,
       12515, 12382, 12225, 12027, 11744, 11168,     0])

In [9]:
pow_t_mean_point_int = pow_t_mean_point.copy()
pow_t_mean_point_int.insert(0, pow_t_mean_point_int[0])

In [10]:
%matplotlib widget
def genera_coppie_tempo_intensita(tempo_array, intensita_array):
    x = []
    y = []
    
    for tempo in range(tempo_array[-1] + 1):
        x.append(tempo)
        if np.isin(tempo, tempo_array):
            indices = np.where(tempo_array == tempo)[0]
            y.append(intensita_array[indices[0]])
        else:
            y.append(y[-1])  # Ripeti l'ultimo valore di intensita_array se tempo non è presente
    
    return np.array(x), np.array(y)

time, power = genera_coppie_tempo_intensita(t_d_int[::-1], pow_t_mean_point_int[::-1])

# Plot dei dati
plt.plot(time, power, marker='o')
plt.xlabel('Tempo')
plt.ylabel('Intensità')
plt.title('Andamento Tempo-Intensità')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Crea un DataFrame con i dati
df = pd.DataFrame({'Tempo': time, 'Valore': power})

# Numero di elementi da considerare per la media mobile
window_size = parseg

# Calcola la media mobile e assegna i risultati al tempo centrale
df['Media_Mobile'] = df['Valore'].rolling(window=window_size, center=True).mean()

# Rimuovi le righe con valori mancanti (prime e ultime finestre)
df = df.dropna()

# Rimuovi i primi neglect_t elementi
df = df.iloc[neglect_t:]

# Stampa il dataframe risultante
print(df)

       Tempo    Valore  Media_Mobile
3250    3250  0.269715      0.269715
3251    3251  0.269715      0.269715
3252    3252  0.269715      0.269715
3253    3253  0.269715      0.269715
3254    3254  0.269715      0.269715
...      ...       ...           ...
32747  32747  0.269715      0.269715
32748  32748  0.269715      0.269715
32749  32749  0.269715      0.269715
32750  32750  0.269715      0.269715
32751  32751  0.269715      0.269715

[29502 rows x 3 columns]


In [21]:
%matplotlib widget
# Crea un oggetto di figura e un singolo assi
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot della media mobile
ax1.plot(df['Tempo'].values, df['Media_Mobile'].values, label='Media Mobile', color='blue')

# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Tempo')
ax1.set_ylabel('Media Mobile', color='blue')
ax1.tick_params('y', colors='blue')

plt.title(f'Media mobile con finestra lunga parseg = {parseg}')
# Mostra il plot
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# simulazione SIGMOIDPULSE

In [13]:
# Inizializza variabile di controllo per il secondo loop
start_second_loop = False

t_d = []
t_d.append(t_mid*2)
# Primo loop
for x in Dd_values[1:-1]:
    # Controlla se è il momento di iniziare il secondo loop
    if x == 0.85:
        start_second_loop = True
        t_d.append((t_mid2 + t_mid)/2)
        continue

    # Calcola i valori in base alla variabile di controllo
    if not start_second_loop:
        y = -np.log((0.1) / (x - 0.85) - 1) / sigm_par + t_mid2
    else:
        y = -np.log((-0.1) / (x - 0.95) - 1) / sigm_par + t_mid

    # Aggiungi il valore calcolato a t_d_int
    t_d.append(y)

t_d.append(t_mid/10 + neglect_t*2.5)

t_d = np.array(t_d)
t_d = np.round(t_d).astype(int)

In [14]:
%matplotlib inline
#funzione Dd
data = np.loadtxt("./build/output/n1/" + function + "_" + t_mid_poiss + ".00_" + sigm_par_poiss + "_0.00_" + Dd + "_1.00_1.00_1.6/" + simulations[0] + "/ext_rateD2.txt").T

data[0] = data[0]/1000
data[1] = data[1]/1.083

#check if periodogram folder exists and if not create one
#spikesim
periodogram_paths = []
s = []
for simulation in simulations:
    periodogram_path = './periodograms/' + Dd.replace('.','_') + '_' + subnets + '_' + function + '_' + simulation
    if not os.path.exists(periodogram_path):  #check if periodogram folder exists and if not create one
        os.makedirs(periodogram_path) 
    periodogram_paths.append(periodogram_path)
    ss = utils.SpikeSim("./build/output/n1/" + function + "_" + t_mid_poiss + ".00_" + sigm_par_poiss + "_0.00_" + Dd + "_1.00_1.00_1.6/" + simulation , 'new_sim_parallel.yaml', neglect_t, np.max(data[0]))
    s.append(ss)

all_pow_t = []
t = []
f = []

for index,ss in enumerate(s):
    output=ss.periodogram(pop=subnets, res=1., N_parseg=parseg, save_img=periodogram_paths[index] + "/periodogram.png")

    f = output[0]
    t = output[1]
    Sxx = output[2]

    mask = (8 <= f) & (f <= 26)
    Sxx_lim = Sxx[mask,:]
    f_lim = f[mask]

    pow_t = []
    for i in range (len(t)):
        pow_t.append(np.mean(Sxx_lim[:,i]))

    all_pow_t.append(pow_t)


np.vstack(all_pow_t)
pow_t_mean = np.mean(all_pow_t, axis=0)
pow_error = np.std(all_pow_t, axis=0)/np.sqrt(len(simulations))

nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250
nparseg = 500	noverlap=250


## time-pow_t

In [18]:
%matplotlib widget
import matplotlib.pyplot as plt

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(9, 5))

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, label='Intensità simulazione', color='orange')
ax1.set_yscale('log')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar((t_d-neglect_t)/1000, pow_t_mean_point, yerr=pow_error_point, fmt='o', label='simulazioni a Dd costante', color='green')

# Plot della media mobile dell'intensità ATTESA
ax1.plot((df['Tempo'].values-neglect_t)/1000, df['Media_Mobile'].values, label='Intensità attesa', color='blue')

# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Tempo [s]')
ax1.set_ylabel('Intensità media')
ax1.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie

plt.title(f'Intensità nel Range \u03B2 per \u03C4 = {1/sigm_par/1000} [s] nparseg = {parseg}')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
def sigmoidpulse(x):
    return 0.95 + 1 / (1 + np.exp(-sigm_par * (x - t_mid +neglect_t))) * (0.85 - 0.95) - 1 / (1 + np.exp(-sigm_par * (x - t_mid2 +neglect_t))) * (0.85 - 0.95) 

d_t = sigmoidpulse(t*1000)
d_t_attesa = sigmoidpulse(df['Tempo'].values-neglect_t)

## dd-pow

In [19]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
# Finestra mobile (rolling window) per il calcolo della moving average
window_size = 1  # Sostituisci con la dimensione della finestra desiderata
moving_avg = np.convolve(pow_t_mean, np.ones(window_size)/window_size, mode='same')

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize = (9,5))

pow_t_mean = np.array(pow_t_mean)

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(d_t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(d_t, pow_t_mean, 'o', label='Intensità media', color='black')
ax1.set_yscale('log')

# Aggiungi la moving average al grafico di ax1
ax1.plot(d_t, moving_avg, color='orange')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar(Dd_values, pow_t_mean_point, yerr=pow_error_point, fmt='o', label='simulazioni a Dd costante', color='green')

# Plot della media mobile dell'intensità ATTESA
ax1.plot(d_t_attesa, df['Media_Mobile'].values, label='Intensità attesa', color='blue')

# Imposta altre personalizzazioni per il tuo grafico, se necessario
plt.xlabel('Dopamine depletion')
plt.ylabel('intensità media')
plt.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie
plt.title(f'Dd vs Intensità nel Range \u03B2 per \u03C4 = {1/sigm_par/1000} [s] nparseg = {parseg}')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# Creazione della maschera per escludere le righe con t < 1 e t > 19
mask = (t >= 1) & (t <= 19)

# Applica la maschera per selezionare solo le righe desiderate
data_to_save = np.column_stack((t[mask], pow_t_mean[mask], d_t[mask], pow_error[mask]))

# Salva i dati in un file di testo
#np.savetxt('./build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg' + str(parseg) + '.txt', data_to_save, header='t pow_t_mean d_t pow_error', delimiter='\t')


# parseg variation

In [18]:
#parseg 250 0verlap 125
Dd_values = [0.85, 0.855, 0.86, 0.865, 0.87, 0.875, 0.88, 0.885, 0.89, 0.895, 0.9, 0.905, 0.91, 0.915, 0.92, 0.925, 0.93, 0.935, 0.94, 0.945, 0.95]
pow_t_mean_point = [0.019914247149334242, 0.020690578678315804, 0.02158271146066814, 0.022364914133701584, 0.023428835457957573, 0.025125497868107482, 0.026346934723841548, 0.028828969225659612, 0.03220798123872726, 0.035254298671665026, 0.03867622848014253, 0.04674626548972391, 0.05068805695941251, 0.06606716969376104, 0.08487862320152097, 0.09270422348347553, 0.1322581180913206, 0.15084846176010766, 0.18747180115646506, 0.22419232383889678, 0.2697145569231209]
pow_error_point = [0.0019838232642017555, 0.001999857551482696, 0.0021708086191144817, 0.0022749744417916227, 0.002441049902399858, 0.002640832617992426, 0.0027738692999452765, 0.0031292618798778368, 0.0036480613158226044, 0.0040683093611075605, 0.00451842724610424, 0.005668445395626774, 0.006082506534680942, 0.007877499532094852, 0.009638645329156629, 0.010410256249777311, 0.012819346780713003, 0.01291999850097032, 0.014568471805002196, 0.015395217530866883, 0.015333769105628054]


In [19]:
# Carica i dati dai file
file_paths = [
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg250.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg500.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg1000.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg2000.txt',
]

data_list = []  # Una lista per memorizzare i dati da ciascun file

for file_path in file_paths:
    data = np.loadtxt(file_path, delimiter='\t', skiprows=1)  # Salta la riga dell'intestazione
    data_list.append(data)

# Estrai le colonne necessarie
t_list = [data[:, 0] for data in data_list]
pow_t_mean_list = [data[:, 1] for data in data_list]
d_t_list = [data[:, 2] for data in data_list]
pow_error_list = [data[:, 3] for data in data_list]

# Fai il plot dei dati con intensità su scala logaritmica
plt.figure(figsize=(10, 6))

for i in range(len(file_paths)):
    parseg_values = [int(file_path.split('parseg')[1].split('.txt')[0]) for file_path in file_paths]
    plt.errorbar(d_t_list[i], pow_t_mean_list[i], yerr=pow_error_list[i], label=f'parseg {parseg_values[i]}', capsize=5)


plt.xlabel('Dopamine depletion')
plt.ylabel('intensità media')
plt.yscale('log')
plt.title('Dd vs intensity changing parseg')
plt.legend()
plt.show()

OSError: ./build/output/n1/sigmoidpulse0_002_13000.00_0.00200_0.00_0.950_1.00_1.00_1.6/parseg250.txt not found.

In [33]:
# Fai il plot dei dati con intensità su scala logaritmica
plt.figure(figsize=(10, 6))

for i in range(len(file_paths)):
    parseg_values = [int(file_path.split('parseg')[1].split('.txt')[0]) for file_path in file_paths]
    plt.errorbar(t_list[i], pow_t_mean_list[i], yerr=pow_error_list[i], label=f'parseg {parseg_values[i]}', capsize=5)

plt.xlabel('tempo (s)')
plt.ylabel('intensità media')
plt.yscale('log')
plt.title('time vs intensity changing parseg')
plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
from scipy.optimize import curve_fit

def linear_fit(parseg, m, q):
    return m * np.array(parseg) + q

# Indice del tempo desiderato
time_index = 80

# Seleziona i valori di intensità al tempo specificato per ciascun parseg
intensity_values = [pow_t_mean_list[i][time_index] for i in range(len(file_paths))]
error_values = [pow_error_list[i][time_index] for i in range(len(file_paths))]

# Fai il fit della retta usando il metodo dei minimi quadrati
params, covariance = curve_fit(linear_fit, parseg_values, intensity_values, sigma=error_values, absolute_sigma=True)

# Estrai i parametri del fit e le relative incertezze
m_fit, q_fit = params
dm, dq = np.sqrt(np.diag(covariance))

# Calcola l'errore totale sulla retta interpolata a parseg=0
error = np.sqrt((m_fit * dm)**2 + dq**2)

# Calcola i valori del fit lineare
fit_values = m_fit * np.array(parseg_values) + q_fit

# Calcola il valore interpolato per parseg = 0 utilizzando l'intercetta del fit lineare
interpolated_value_at_parseg_0 = q_fit
# Calcola l'errore totale sulla retta interpolata a parseg=0
error = np.sqrt((m_fit * dm)**2 + dq**2)

# Aggiungi un punto a parseg=0
parseg_values_extended = np.insert(parseg_values, 0, 0)
fit_values_extended = np.insert(fit_values, 0, q_fit)

# Fai il plot dei dati, della retta di fit e del valore interpolato per parseg = 0
plt.figure(figsize=(8, 5))
plt.errorbar(parseg_values, intensity_values, yerr=error_values, fmt='o', capsize=5, label='intensità variando parseg')
plt.plot(parseg_values_extended, fit_values_extended, '-', label='Fit lineare')
plt.errorbar([0], [interpolated_value_at_parseg_0], color='red', yerr=error, fmt='o', capsize=5, label='parseg = 0')
plt.xlabel('parseg')
plt.ylabel('Intensità media')
plt.title(f'Fit lineare ai dati al tempo {t_list[0][time_index]:.2f} s')
plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# parseg 0

In [35]:
from scipy.optimize import curve_fit

def linear_fit(parseg, m, q):
    return m * np.array(parseg) + q

# Creazione di una lista per memorizzare i valori estrapolati per parseg=0 per ogni valore di t
values_at_parseg_0 = []
errors_at_parseg_0 = []

# Itera su tutti i valori di t
for specific_t_value in t_list[0]:
    specific_t_index = np.argmin(np.abs(t_list[0] - specific_t_value))

    intensity_at_specific_t = [pow_t_mean_list[i][specific_t_index] for i in range(len(file_paths))]
    error_at_specific_t = [pow_error_list[i][specific_t_index] for i in range(len(file_paths))]

    # Estrai i valori di parseg
    parseg_values = [int(file_path.split('parseg')[1].split('.txt')[0]) for file_path in file_paths]
    parseg_values = np.array(parseg_values, dtype=float)

    # Ordina i dati in base ai valori di parseg
    sorted_data = sorted(zip(parseg_values, intensity_at_specific_t, error_at_specific_t))
    parseg_values, intensity_at_specific_t, error_at_specific_t = zip(*sorted_data)

    # Fai il fit della retta usando il metodo dei minimi quadrati
    params, covariance = curve_fit(linear_fit, parseg_values, intensity_at_specific_t, sigma=error_at_specific_t, absolute_sigma=True)

    # Estrai i parametri del fit e le relative incertezze
    m_fit, q_fit = params
    dm, dq = np.sqrt(np.diag(covariance))

    # Calcola l'errore totale sulla retta interpolata a parseg=0
    errors = np.sqrt((m_fit * dm)**2 + dq**2)

    # Aggiungi il valore interpolato e l'errore alle rispettive liste
    values_at_parseg_0.append(linear_fit(0, m_fit, q_fit))
    errors_at_parseg_0.append(errors)

# Plot dei valori interpolati a parseg=0 nel tempo con errori
plt.figure(figsize=(10, 6))
plt.errorbar(t_list[0], values_at_parseg_0, yerr=errors_at_parseg_0, marker='o', linestyle='-', color='green', label='Interpolati a parseg=0')
plt.xlabel('Tempo (t)')
plt.ylabel('Valori interpolati a parseg=0')
plt.title('Plot di Valori interpolati a parseg=0 nel tempo con errori (retta)')
plt.legend()
plt.grid(True)
plt.show()

# Salva i dati in un file di testo
#data_to_save = np.column_stack((t_list[0], values_at_parseg_0, d_t_list[0], errors_at_parseg_0))
#np.savetxt('./build/output/n1/' + function + '_" + t_mid_poiss + ".00_" + sigm_par_poiss + "_0.00_' + Dd + '_1.00_1.00_1.6/parseg0.txt', data_to_save, header='t pow_t_mean d_t pow_error', delimiter='\t')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
# Carica i dati dai file
file_paths = [
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg0.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg250.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg500.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg1000.txt',
    './build/output/n1/' + function + '_' + t_mid_poiss + '.00_' + sigm_par_poiss + '_0.00_' + Dd + '_1.00_1.00_1.6/parseg2000.txt',
]

data_list = []  # Una lista per memorizzare i dati da ciascun file

# Estrai i valori di parseg una sola volta fuori dal loop
parseg_values = [int(file_path.split('parseg')[1].split('.txt')[0]) for file_path in file_paths]

for file_path, parseg_value in zip(file_paths, parseg_values):
    data = np.loadtxt(file_path, delimiter='\t', skiprows=1)  # Salta la riga dell'intestazione
    data_list.append(data)

# Estrai le colonne necessarie
t_list = [data[:, 0] for data in data_list]
pow_t_mean_list = [data[:, 1] for data in data_list]
d_t_list = [data[:, 2] for data in data_list]
pow_error_list = [data[:, 3] for data in data_list]

# Fai il plot dei dati con intensità su scala logaritmica
plt.figure(figsize=(10, 6))

for i in range(len(file_paths)):
    # Usa il colore nero solo quando parseg è uguale a 0
    if parseg_values[i] == 0:
        plt.errorbar(d_t_list[i], pow_t_mean_list[i], yerr=pow_error_list[i], label=f'parseg {parseg_values[i]}', capsize=5, color='black')
    else:
        plt.errorbar(d_t_list[i], pow_t_mean_list[i], yerr=pow_error_list[i], label=f'parseg {parseg_values[i]}', capsize=5)

plt.xlabel('Dopamine depletion')
plt.ylabel('intensità media')
plt.yscale('log')
plt.title('Dd vs intensity changing parseg')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
# Fai il plot dei dati con intensità su scala logaritmica
plt.figure(figsize=(10, 6))

for i in range(len(file_paths)):
    parseg_values = [int(file_path.split('parseg')[1].split('.txt')[0]) for file_path in file_paths]

    # Usa il colore nero solo quando parseg è uguale a 0
    if parseg_values[i] == 0:
        plt.errorbar(t_list[i], pow_t_mean_list[i], yerr=pow_error_list[i], label=f'parseg {parseg_values[i]}', capsize=5, color='black')
    else:
        plt.errorbar(t_list[i], pow_t_mean_list[i], yerr=pow_error_list[i], label=f'parseg {parseg_values[i]}', capsize=5)

plt.xlabel('tempo (s)')
plt.ylabel('intensità media')
plt.yscale('log')
plt.title('time vs intensity changing parseg')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
%matplotlib widget

# Creare un grafico dei valori nel tempo
# Finestra mobile (rolling window) per il calcolo della moving average
window_size = 1  # Sostituisci con la dimensione della finestra desiderata
moving_avg = np.convolve(pow_t_mean, np.ones(window_size)/window_size, mode='same')

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(9, 5))

pow_t_mean = np.array(pow_t_mean)

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, label='Intensità media', color='orange')
ax1.set_yscale('log')

# Aggiungi la moving average al grafico di ax1
ax1.plot(t, moving_avg, label=f'Moving Average ({window_size}-point)', color='orange')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar(t_d/1000, pow_t_mean_point, yerr=pow_error_point, fmt='o', label='simulazioni a Dd costante', color='green')

# Imposta altre personalizzazioni per il tuo grafico, se necessario
plt.xlabel('Tempo [s]')
plt.ylabel('Intensità media')
plt.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie

plt.title(f'Time Series della Media dell\'Intensità nel Range \u03B2 per \u03C4 = {1/sigm_par/1000} [s] nparseg = {parseg}')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# confronto dd costante e dinamica

In [39]:
%matplotlib widget

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(9, 5))

pow_t_mean = np.array(pow_t_mean)

ax1.set_yscale('log')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar(t_list[0], pow_t_mean_list[0], yerr=pow_error_list[0], label=f'parseg {parseg_values[0]}', capsize=5, color='black')
ax1.errorbar(t_d/1000, pow_t_mean_point, yerr=pow_error_point, fmt='o', label='simulazioni a Dd costante', color='green')
# Imposta altre personalizzazioni per il tuo grafico, se necessario
plt.xlabel('Tempo [s]')
plt.ylabel('Intensità media')
plt.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie

plt.title(f'Time Series della Media dell\'Intensità nel Range \u03B2 per \u03C4 = {1/sigm_par/1000} [s]')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
# Finestra mobile (rolling window) per il calcolo della moving average
window_size = 1  # Sostituisci con la dimensione della finestra desiderata
moving_avg = np.convolve(pow_t_mean, np.ones(window_size)/window_size, mode='same')

# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize = (9,5))

pow_t_mean = np.array(pow_t_mean)

ax1.set_yscale('log')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
ax1.errorbar(d_t_list[0], pow_t_mean_list[0], yerr=pow_error_list[0], label=f'parseg {parseg_values[0]}', capsize=5, color='black')
ax1.errorbar(Dd_values, pow_t_mean_point, yerr=pow_error_point, fmt='o', label='simulazioni a Dd costante', color='green')

# Imposta altre personalizzazioni per il tuo grafico, se necessario
plt.xlabel('Dopamine depletion')
plt.ylabel('intensità media')
plt.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie
plt.title(f'Dd vs Intensità nel Range \u03B2 per \u03C4 = {1/sigm_par/1000} [s]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …